In [107]:
import numpy as np
import networkx as nx
import pandas as pd
import csv

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from neo4j import GraphDatabase
from getpass import getpass

Db usato mapping_path_zona

In [108]:
user = getpass('Neo4j user: ')
password = getpass('Neo4j pass: ')

Neo4j user: ········
Neo4j pass: ········


In [109]:
uri = 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri, auth=('neo4j','password'), encrypted=False)

In [110]:
def cypherQueryToDataFrame(query):
    with driver.session(database = 'neo4j') as session:
        result = session.run(query)
        return pd.DataFrame(result.data(), columns=result.keys())

In [111]:
query = """
MATCH (p:Point)
Where p.coordinates is not null
RETURN p.latitude as latitude, p.longitude as longitude, id(p) as idPoint
"""

df = cypherQueryToDataFrame(query)
df

,latitude,longitude,idPoint
0,44.405436,8.942695,246
1,44.405521,8.942335,248
2,44.406121,8.942889,256
3,44.406231,8.943164,257
4,44.406301,8.942762,258
...,...,...,...
7489,44.452345,9.017467,9224
7490,44.427067,8.833002,9225
7491,44.426993,8.832645,9226
7492,44.426776,8.833127,9227


In [112]:
coordinate_punti = []
# Creo punti
for index, row in df.iterrows():
    coordinate_punti.append(Point(row['longitude'],row['latitude']))

In [114]:
zone = """
MATCH (z:Zona)-[]->(g:Geometry)
WITH DISTINCT g.linesCoordinates as lines, z
UNWIND lines as line
RETURN z.descZona as nameZona, line.latitude as lat, line.longitude as long
"""

df1 = cypherQueryToDataFrame(zone)
df1

,nameZona,lat,long
0,CREVARI,44.436638,8.696006
1,CREVARI,44.436649,8.696012
2,CREVARI,44.436727,8.696031
3,CREVARI,44.436812,8.696039
4,CREVARI,44.436858,8.696037
...,...,...,...
133305,Zoagli,44.347351,9.242179
133306,Zoagli,44.347365,9.242125
133307,Zoagli,44.347398,9.242074
133308,Zoagli,44.347444,9.242020


In [116]:
zone_dict = {}
for index, row in df1.iterrows():
    zone = row['nameZona']
    lat = row['lat']
    long = row['long']
    if zone in zone_dict:
        zone_dict[zone].append((long, lat))
    else:
        zone_dict[zone] = [(long, lat)]

Eseguo la stessa operazione per i punti che si trovano in single polygon 

In [115]:
zoneSingleP = """
MATCH (z:Zona)-[]->(g:Geometry)-[]->(s:SinglePolygon)
WITH DISTINCT s.linesCoordinates as lines, z
UNWIND lines as line
RETURN z.descZona as nameZona, line.latitude as lat, line.longitude as long
"""

zoneSingleP = cypherQueryToDataFrame(zoneSingleP)
zoneSingleP

,nameZona,lat,long
0,PALMARO,44.416138,8.769872
1,PALMARO,44.416102,8.770102
2,PALMARO,44.415906,8.770041
3,PALMARO,44.415373,8.773528
4,PALMARO,44.415565,8.773590
...,...,...,...
2352,SAMPIERDARENA,44.405175,8.879549
2353,SAMPIERDARENA,44.405835,8.875861
2354,SAMPIERDARENA,44.408738,8.876870
2355,SAMPIERDARENA,44.408778,8.876643


In [122]:
for index, row in zoneSingleP.iterrows():
    zone = row['nameZona']
    lat = row['lat']
    long = row['long']
    if zone in zone_dict:
        zone_dict[zone].append((long, lat))
    else:
        zone_dict[zone] = [(long, lat)]

In [123]:
for i in zone_dict:
    zone_dict[i] = Polygon(zone_dict[i])

In [124]:
len(zone_dict)

121

In [127]:
mapping_zone_point = []
for point in coordinate_punti:
    for i in zone_dict:
        if point.within(zone_dict[i]):
            mapping_zone_point.append(i + "," + str(point))
            

In [128]:
len(mapping_zone_point)

7493

In [129]:
mapping_zone_point

['S.VINCENZO,POINT (8.9426947 44.4054364)',
 'S.VINCENZO,POINT (8.9423352 44.4055207)',
 'S.VINCENZO,POINT (8.9428886 44.4061213)',
 'S.VINCENZO,POINT (8.9431643 44.406231)',
 'S.VINCENZO,POINT (8.9427624 44.4063012)',
 'S.VINCENZO,POINT (8.9434776 44.4081443)',
 'S.VINCENZO,POINT (8.9427278 44.408284)',
 'MADDALENA,POINT (8.9300571 44.4132844)',
 'MADDALENA,POINT (8.9339091 44.4115772)',
 'MADDALENA,POINT (8.9302867 44.4134113)',
 "PRE',POINT (8.9288503 44.4135558)",
 'S.VINCENZO,POINT (8.9364845 44.4066589)',
 'S.VINCENZO,POINT (8.9363413 44.406382)',
 'S.VINCENZO,POINT (8.9374767 44.4064097)',
 'S.VINCENZO,POINT (8.9357598 44.4053952)',
 'S.VINCENZO,POINT (8.9357128 44.4053367)',
 'CARIGNANO,POINT (8.9365033 44.4050799)',
 'S.VINCENZO,POINT (8.9346372 44.4063648)',
 'BRIGNOLE,POINT (8.9436669 44.403394)',
 'BRIGNOLE,POINT (8.9440432 44.404082)',
 'BRIGNOLE,POINT (8.9443874 44.4047315)',
 'S.VINCENZO,POINT (8.9433805 44.4042651)',
 'S.VINCENZO,POINT (8.940642 44.4042049)',
 'S.VINCEN

In [130]:
Zona = []
Point = []
for dato in mapping_zone_point:
    parte1, parte2 = dato.split(',')
    Zona.append(parte1.strip())
    Point.append(parte2.strip())

# Creazione del dataframe
df = pd.DataFrame({'Zona': Zona, 'Point': Point})

df

,Zona,Point
0,S.VINCENZO,POINT (8.9426947 44.4054364)
1,S.VINCENZO,POINT (8.9423352 44.4055207)
2,S.VINCENZO,POINT (8.9428886 44.4061213)
3,S.VINCENZO,POINT (8.9431643 44.406231)
4,S.VINCENZO,POINT (8.9427624 44.4063012)
...,...,...
7488,PRATO,POINT (9.0174669 44.4523455)
7489,MULTEDO,POINT (8.8330017 44.4270671)
7490,MULTEDO,POINT (8.8326455 44.4269927)
7491,MULTEDO,POINT (8.8331272 44.4267759)


In [131]:
df.to_csv("mapping_zona_point.csv", index=False)